<a href="https://colab.research.google.com/github/Jwilson1172/lambda/blob/master/unit-1-build/clean_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#used to change all the relive dates to datetimes
import datetime

# this in important because the time stamps in the dataset are relitive to when they were scraped
DATE_SCRAPED="02-24-2020"

In [0]:
# read in the data frame that has my scrapped content
df=pd.read_csv('/content/google_reviews_chanellos.csv')
df_labels=pd.read_csv('/content/chanellos_locations.csv')


# **EDA**

In [3]:
# inspect head @TODO DROP NAME COL
print(df.shape)
df.head()

(2096, 7)


,web-scraper-order,web-scraper-start-url,number_to_rate,name,published_time,review_content,rating_html
0,1582527995-344,https://www.google.com/maps/place/Chanello's+P...,124 reviews,lauren sams,2 weeks ago,Great and fast service with excellent food.,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
1,1582527995-428,https://www.google.com/maps/place/Chanello's+P...,124 reviews,Krys Peirce,a year ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
2,1582527918-244,https://www.google.com/maps/place/Chanello's+P...,101 reviews,lori kent,3 years ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
3,1582528346-1348,https://www.google.com/maps/place/Chanello's+P...,62 reviews,Christian Betz,4 years ago,NaN,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."
4,1582527995-377,https://www.google.com/maps/place/Chanello's+P...,124 reviews,Janel Brosseit,a year ago,This rating goes more towards the food. Just r...,"<span jstcache=""293"" jsinstance=""0"" class=""sec..."


In [11]:
# a correction to the number of reviews from the time that the data
# was scrapped and the labels dataset was scraped
df_labels['num_to_rate'].at[13]=101

df_labels.head(20)

,web-scraper-order,name,rating,address,num_to_rate,start_url
0,1582755020-11,Chanello's Pizza #1,3.7,8245 Hampton Blvd,87,https://www.google.com/maps/place/Chanello's+P...
1,1582755020-2,Chanello's Pizza #19,4.0,2226 Pleasure House Rd,68,https://www.google.com/maps/place/Chanello's+P...
2,1582755020-16,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,https://www.google.com/maps/place/Chanello's+P...
3,1582755020-1,Chanello's Pizza #2,3.1,3201 E Ocean View Ave,116,https://www.google.com/maps/place/Chanello's+P...
4,1582755020-6,Chanello's Pizza #11,3.4,6657 Indian River Rd,148,https://www.google.com/maps/place/Chanello's+P...
5,1582755020-7,Chanello's Pizza #7,3.5,5300 Kemps River Dr #118,63,https://www.google.com/maps/place/Chanello's+P...
6,1582755020-19,Chanello's Pizza #26,2.9,1818 Todds Ln,141,https://www.google.com/maps/place/Chanello's+P...
7,1582755020-9,Chanello's Pizza #5,3.3,901 W 21st St,157,https://www.google.com/maps/place/Chanello's+P...
8,1582755020-14,Chanello's Pizza #8,3.5,2129 General Booth Blvd #107,64,https://www.google.com/maps/place/Chanello's+P...
9,1582755020-13,Chanello's Pizza #32,3.4,3200 High St,101,https://www.google.com/maps/place/Chanello's+P...


In [10]:
# define a function that cleans up the number to rate column of the
# reviews dataset by removing ' reviews' from the end of the cell content
def strip_n2r(x):
  if isinstance(x,float):
    return x
  else:
    return x[:-8]
# make sure that i didnt break anything
test=df['number_to_rate'].apply(strip_n2r)
test.head()

0    124
1    124
2    101
3     62
4    124
Name: number_to_rate, dtype: object

In [0]:
# make changes to df
df['number_to_rate']=df['number_to_rate'].apply(strip_n2r)

#Data Cleaning/feature engineering

###**geo-codes engineering**

In [17]:
# im going to have to extract the geo code from the following links and compare them to my geo codes from
# my labels df to map the reviews to a particular store
print("labels df:")
print(df_labels['start_url'].iloc[0])
print("\nreviews df:")
df['web-scraper-start-url'].iloc[0]

labels df:
https://www.google.com/maps/place/Chanello's+Pizza+%231/@36.928009,-76.3205357,17z/data=!3m1!4b1!4m10!1m2!2m1!1schnello's+pizza!3m6!1s0x89ba997fe547e42f:0x90f52467e57e80a4!8m2!3d36.928009!4d-76.318347!9m1!1b1

reviews df:


"https://www.google.com/maps/place/Chanello's+Pizza+%2333/@36.7819861,-76.3383989,17z/data=!3m1!4b1!4m10!1m2!2m1!1sChanello's+Pizza!3m6!1s0x89baa32fc8a20eb5:0x27c72a7bc0bad400!8m2!3d36.7819818!4d-76.3362102!9m1!1b1"

In [18]:
# define a function to strip the url strings to reveal geo tags
def parse_geo(x):
  return x.split('@')[1].split(',')[:-1]
# test it out before i make any changes
test=df['web-scraper-start-url'].apply(parse_geo)
test.head()

0    [36.7819861, -76.3383989]
1    [36.7819861, -76.3383989]
2    [36.8262133, -76.4123802]
3    [36.8031384, -76.1770067]
4    [36.7819861, -76.3383989]
Name: web-scraper-start-url, dtype: object

In [0]:
# make changes
df['geo']=df['web-scraper-start-url'].apply(parse_geo)
df_labels['geo']=df_labels['start_url'].apply(parse_geo)

str

###**rating valuation engineering**

In [23]:
# on the reviews it had images of lit stars triggered by js
# to work around this i took the html source because i knew it would
# show which stars are supossed to be triggered, but as a drawback
# no i have to clean HTML instead of pretty output
df['rating_html'].head()

0    <span jstcache="293" jsinstance="0" class="sec...
1    <span jstcache="293" jsinstance="0" class="sec...
2    <span jstcache="293" jsinstance="0" class="sec...
3    <span jstcache="293" jsinstance="0" class="sec...
4    <span jstcache="293" jsinstance="0" class="sec...
Name: rating_html, dtype: object

In [24]:
# make a funtion to apply to col
def clean_rating(x):
  # make a list of strings to work with
  out=x.split()
  # sort list so that duplicates group togeather
  out.sort()
  # the first element is just '<span>' and the following 20 elements are redundant,
  # the last 5 elements are placment flags
  # remove all of them and leave me with just the number of active stars
  # then get a count so i can have a pretty int
  return len(out[21:len(out)-5])

# test to make sure that it works
test=df['rating_html'].apply(clean_rating)
test.head()

0    5
1    1
2    4
3    4
4    2
Name: rating_html, dtype: int64

In [0]:
# make changes to df
df['rating_html']=df['rating_html'].apply(clean_rating)

### **datetime engeenering for graphing**

In [26]:
# humanfied dates are great to read, not so great to graph, i need to make theese
# back into dt format,(why i have a glob at the top with the date of scrape)
df['published_time'].value_counts()

a year ago       628
2 years ago      394
3 years ago      168
7 months ago      91
8 months ago      90
6 months ago      74
4 years ago       72
11 months ago     68
5 months ago      67
9 months ago      65
10 months ago     56
a month ago       55
4 months ago      54
3 months ago      47
2 months ago      35
5 years ago       26
a week ago        18
2 weeks ago       17
6 years ago       16
3 weeks ago       12
4 weeks ago       11
8 years ago       10
7 years ago        9
5 days ago         4
a day ago          4
6 days ago         2
9 years ago        2
3 days ago         1
Name: published_time, dtype: int64

In [0]:
# create a map for all units to hours
time_map={'second':0.000278,'minute':0.0167,'hour':1,'day':24,'week':168,'month':530.5,'year':8766}
# make a function that takes a relitive time 'a day ago' and transfers that to a datetime
def human_to_dt(x):
  # the number that i will end up subtracting from scrape date
  diff=0
  # make a datetime object to hold the date that the dataset was scrapped
  dt=datetime.datetime.strptime(DATE_SCRAPED, '%m-%d-%Y')
  # strip the string down to two values a quantifier and a unit
  q,u=x[:-3].strip().split(' ')
  # remove trailing s on unit it is not needed
  if u[len(u)-1]=="s":
    u=u[:len(u)-1]
  # check if there is just one unit if so then set diff=1
  if q == "a":
    diff=time_map[u]
  # if the number is not one then multiply the q by the map_key entry for u
  else:
    #safe cast
    try:
      diff=time_map[u]*int(q)
    except:
      print("ERROR")
      return 0
  # convert dt to utc timestamp
  dt=dt-datetime.timedelta(hours=diff)
  timestamp = dt.replace(tzinfo=datetime.timezone.utc).timestamp()
  return timestamp

In [32]:
# making sure that it works how i want it to
testvalue=df['published_time'].iloc[0]
print(human_to_dt(testvalue))
del testvalue

1581292800.0


In [0]:
# apply changes to df
df['published_time']=df['published_time'].apply(human_to_dt)

## more data cleaning to drop columns that i dont need anymore

In [34]:
df.head()

,web-scraper-order,web-scraper-start-url,number_to_rate,name,published_time,review_content,rating_html,geo
0,1582527995-344,https://www.google.com/maps/place/Chanello's+P...,124,lauren sams,1.581293e+09,Great and fast service with excellent food.,5,"[36.7819861, -76.3383989]"
1,1582527995-428,https://www.google.com/maps/place/Chanello's+P...,124,Krys Peirce,1.550945e+09,NaN,1,"[36.7819861, -76.3383989]"
2,1582527918-244,https://www.google.com/maps/place/Chanello's+P...,101,lori kent,1.487830e+09,NaN,4,"[36.8262133, -76.4123802]"
3,1582528346-1348,https://www.google.com/maps/place/Chanello's+P...,62,Christian Betz,1.456272e+09,NaN,4,"[36.8031384, -76.1770067]"
4,1582527995-377,https://www.google.com/maps/place/Chanello's+P...,124,Janel Brosseit,1.550945e+09,This rating goes more towards the food. Just r...,2,"[36.7819861, -76.3383989]"


In [35]:
df_labels.head()

,web-scraper-order,name,rating,address,num_to_rate,start_url,geo
0,1582755020-11,Chanello's Pizza #1,3.7,8245 Hampton Blvd,87,https://www.google.com/maps/place/Chanello's+P...,"[36.928009, -76.3205357]"
1,1582755020-2,Chanello's Pizza #19,4.0,2226 Pleasure House Rd,68,https://www.google.com/maps/place/Chanello's+P...,"[36.9056756, -76.1367491]"
2,1582755020-16,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,https://www.google.com/maps/place/Chanello's+P...,"[36.7819818, -76.3383989]"
3,1582755020-1,Chanello's Pizza #2,3.1,3201 E Ocean View Ave,116,https://www.google.com/maps/place/Chanello's+P...,"[36.932126, -76.2046187]"
4,1582755020-6,Chanello's Pizza #11,3.4,6657 Indian River Rd,148,https://www.google.com/maps/place/Chanello's+P...,"[36.819091, -76.2241417]"


In [0]:
# going to drop internal scraper columns that dont add context for the information
# that is provided by them
df=df.drop(['web-scraper-order','web-scraper-start-url'],axis=1)
df_labels=df_labels.drop(['web-scraper-order','start_url'],axis=1)

##merging df's based on geo codes

In [45]:
# but first i have to cast and round existing geo codes in both df's because
# google does such a great job at consitency
testvalue=df['geo'].iloc[0]
# define a function that reurns a tuple of rounded floates from the geo tags
def clean_geos(x):
  out=[round(float(x[0]),2),round(float(x[1]),2)]
  return out

# self explainitory
def sep_lat(x):
  return x[0]
def sep_long(x):
  return x[1]

print(clean_geos(testvalue))
print(sep_lat(testvalue))
print(sep_long(testvalue))

[36.78, -76.34]
36.7819861
-76.3383989
